In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import torch
from src.data_processing.eda_utils import show_3d_plot_shapenet
import os
import json
from src.model.PTv1.model import PointTransformerSegmentation
import torch.nn as nn

In [3]:
test_pc = "02691156/8add45a11c9fcb446eb5821e78d8898a.txt"
path_to_data = "../../data/shapenetcore_partanno_segmentation_benchmark_v0_normal/"
point_cloud_path = os.path.join(path_to_data, test_pc)
point_clouds = pd.read_csv(point_cloud_path,
                            delimiter=" ",
                            names = ["x", "y", "z", "r", "g", "b", "label"],
                            header=None).to_numpy().astype('float32')
point_clouds_original = pd.read_csv(point_cloud_path,
                            delimiter=" ",
                            names = ["x", "y", "z", "r", "g", "b", "label"],
                            header=None)
point_clouds = torch.Tensor(point_clouds[:, :3]).unsqueeze(0)

In [4]:
point_clouds.shape

torch.Size([1, 2774, 3])

In [6]:
model = PointTransformerSegmentation(npoints = 1000, n_classes = 4, in_dim = 3)
model.load_state_dict(torch.load("../model_weights/shapenet_airplane_model_weights_segmentation_final.pt", map_location=torch.device('cpu')))
model.eval()
"Model Loaded Successfully"

'Model Loaded Successfully'

In [7]:
out_pos, out = model(point_clouds)
out = nn.Softmax(dim=1)(out)
out = torch.argmax(out, dim=2)

In [8]:
inferenced_points = torch.cat((point_clouds, out.unsqueeze(1).permute(0, 2, 1)), dim = 2)
inferenced_points_df = pd.DataFrame(inferenced_points.squeeze(0).detach().numpy(), columns = ["x", "y", "z", "label"])

In [9]:
show_3d_plot_shapenet(inferenced_points_df)

In [10]:
show_3d_plot_shapenet(point_clouds_original)